# Analysis Development
 - Given the user has access to Spotify, this script will return a simple plot of their liked songs grouped by day and number of songs liked per day
 - Additional updated research will be appended in here

##### Import the modules

In [10]:
import numpy as np
import pandas as pd

import spotipy
from spotipy import SpotifyOAuth

##### Client ID and Secret are passed from an external source

In [11]:
import settings

In [12]:
# Client ID and Client Secret is called and used
spotify_client_id = settings.SPOTIFY_CLIENT_ID
spotify_client_secret = settings.SPOTIFY_CLIENT_SECRET
spotify_redirect_uri = "http://localhost:5000/callback"

##### With the application setup, we just need to call in the authenticator and method itself
 - This will vary based on the type of access. For me, it is suitable.

In [25]:
# provide scope of access for Python which requests access from the current user
scope =  'user-library-read user-top-read user-read-private playlist-read-private playlist-read-collaborative playlist-modify-public playlist-modify-private'

# authentication manager ensures secured point of access from spotify to python to client (me)
auth_manager = SpotifyOAuth(
                            scope = scope, 
                            client_id = spotify_client_id,
                            client_secret = spotify_client_secret,
                            redirect_uri = spotify_redirect_uri
                            )

# initial call of spotify using authentication manager
sp = spotipy.Spotify(auth_manager = auth_manager)

##### Test connection (expectancy of a dict response with personal info)

In [22]:
sp.current_user()

##### Users Top Tracks

In [23]:
sp.current_user_top_tracks()

{'items': [{'album': {'album_type': 'SINGLE',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4iMO20EPodreIaEl8qW66y'},
      'href': 'https://api.spotify.com/v1/artists/4iMO20EPodreIaEl8qW66y',
      'id': '4iMO20EPodreIaEl8qW66y',
      'name': 'Still Woozy',
      'type': 'artist',
      'uri': 'spotify:artist:4iMO20EPodreIaEl8qW66y'}],
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
     'IE',
     'IT',
     'LV',
     'LT',
     'LU',
     'MY',
     'MT',
     'MX',
     'NL',
     'NZ',
     'NI',
     'NO',
     'PA',
     'PY',
     'PE',
     'PH',
     'PL',
     'PT',
     'SG',
     'SK',
     'ES',
     'SE',
     'CH',
     'TW',
     'TR',
     'UY',
     'US',
     'GB',
 

##### While loop to collect all songs in my liked

In [26]:
liked_songs = []
offset = 0
users_liked = sp.current_user_saved_tracks(limit = 50)

while users_liked["next"] != None:
    liked_songs.extend(users_liked["items"])
    offset+=50
    users_liked = sp.current_user_saved_tracks(offset=offset,limit = 50)

liked_songs.extend(users_liked["items"])

##### Observed collection of my liked songs 

In [28]:
len(liked_songs)

1796

In [35]:
df = pd.DataFrame(list(map(lambda x: x["track"],liked_songs)))

In [40]:
df2 = pd.DataFrame(liked_songs)

In [43]:
df["added_at"] = pd.to_datetime(df2["added_at"]).dt.strftime('%Y-%m-%dT%H:%M:%SZ')


In [58]:
# df["release_date"] = df["album"].apply(lambda x: x["release_date"])
df["release_date"] = pd.to_datetime(df["album"].apply(lambda x: x["release_date"]), format='ISO8601').dt.strftime('%Y-%m-%d')

In [59]:
df[["added_at","release_date"]]

,added_at,release_date
0,2024-08-15T18:53:43Z,2024-08-05
1,2024-08-15T18:51:26Z,2024-08-09
2,2024-07-18T19:15:41Z,2003-11-19
3,2024-07-18T19:04:00Z,2024-05-23
4,2024-06-24T16:15:38Z,2020-11-20
...,...,...
1791,2014-09-09T01:18:41Z,2014-08-12
1792,2014-09-05T01:54:20Z,2014-07-29
1793,2014-07-08T02:09:43Z,2014-05-26
1794,2014-07-07T07:55:32Z,2011-08-18
